In [2]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [3]:
from openai import OpenAI
client = OpenAI()

# Function to get the vector embedding for a given text
def get_vector_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    embeddings = [r.embedding for r in response.data]
    return embeddings[0]

get_vector_embeddings("Your text string goes here")

[-0.007611386943608522,
 -0.005377683322876692,
 0.011407003737986088,
 -0.024762200191617012,
 -0.02490999363362789,
 0.039796870201826096,
 -0.010325420647859573,
 -0.009411785751581192,
 -0.013294734992086887,
 -0.009841731749475002,
 -0.011736180633306503,
 0.008229434490203857,
 -0.014295702800154686,
 0.007765898481011391,
 0.01000296138226986,
 -0.004937660414725542,
 0.02282744273543358,
 -0.0016215344658121467,
 0.015330259688198566,
 -0.010258241556584835,
 0.004890635143965483,
 0.012240023352205753,
 0.004742841236293316,
 0.010547111742198467,
 -0.006593624129891396,
 -0.0003955166903324425,
 0.005703501868993044,
 -0.012811045162379742,
 0.016364818438887596,
 0.004437176510691643,
 0.006371933035552502,
 -0.006798520218580961,
 -0.015303388237953186,
 -0.006170396227389574,
 -0.018178651109337807,
 0.0037049248348921537,
 0.0036108740605413914,
 -0.019441617652773857,
 0.030311189591884613,
 -0.006953032221645117,
 0.00816897302865982,
 0.009492400102317333,
 -0.00093126

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,  # 100 tokens
    chunk_overlap=20,  # 20 tokens of overlap
)

text = """
Welcome to the "Unicorn Enterprises: Where Magic Happens" Employee Handbook! We're thrilled to have you join our team of dreamers, doers, and unicorn enthusiasts. At Unicorn Enterprises, we believe that work should be as enchanting as it is productive. This handbook is your ticket to the magical world of our company, where we'll outline the principles, policies, and practices that guide us on this extraordinary journey. So, fasten your seatbelts and get ready to embark on an adventure like no other!
Certainly, here are five middle paragraphs for your fake employee handbook:

**1: Our Magical Culture**

At Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don't be surprised if you find a fairy tale book in the breakroom or a gnome guiding you to the restroom. Our culture is designed to spark your imagination and encourage collaboration among our magical team.

**2: Unicorn Code of Conduct**

While we embrace creativity, we also value professionalism. Our Unicorn Code of Conduct ensures that we maintain a harmonious and respectful environment. Treating all team members, regardless of their unicorn species, with kindness and respect is essential. We also encourage open communication and constructive feedback because, in our world, every opinion matters, just like every horn on a unicorn's head!

**3: Magical Work-Life Balance**

At Unicorn Enterprises, we understand the importance of maintaining a balanced life. We offer flexible work hours, magical mental health days, and even an on-site wizard to provide stress-relief spells when needed. We believe that a happy and well-rested employee is a creative and productive employee. So, don't hesitate to use our relaxation chambers or join a group meditation session under the office rainbow.

**4: Enchanted Benefits**

Our commitment to your well-being extends to our magical benefits package. You'll enjoy a treasure chest of perks, including unlimited unicorn rides, a bottomless cauldron of coffee and potions, and access to our company library filled with spellbinding books. We also offer competitive health and dental plans, ensuring your physical well-being is as robust as your magical spirit.

**5: Continuous Learning and Growth**

At Unicorn Enterprises, we believe in continuous learning and growth. We provide access to a plethora of online courses, enchanted workshops, and wizard-led training sessions. Whether you're aspiring to master new spells or conquer new challenges, we're here to support your personal and professional development.

As we conclude this handbook, remember that at Unicorn Enterprises, the pursuit of excellence is a never-ending quest. Our company's success depends on your passion, creativity, and commitment to making the impossible possible. We encourage you to always embrace the magic within and outside of work, and to share your ideas and innovations to keep our enchanted journey going. Thank you for being a part of our mystical family, and together, we'll continue to create a world where magic and business thrive hand in hand!
"""

chunks = text_splitter.split_text(text=text)
print(chunks[0:3])

['Welcome to the "Unicorn Enterprises: Where Magic Happens" Employee Handbook! We\'re thrilled to have you join our team of dreamers, doers, and unicorn enthusiasts. At Unicorn Enterprises, we believe that work should be as enchanting as it is productive. This handbook is your ticket to the magical world of our company, where we\'ll outline the principles, policies, and practices that guide us on this extraordinary journey. So, fasten your seatbelts and get ready', 'practices that guide us on this extraordinary journey. So, fasten your seatbelts and get ready to embark on an adventure like no other!', 'Certainly, here are five middle paragraphs for your fake employee handbook:']


In [6]:
import numpy as np
import faiss

# Get vector embeddings for the chunks from last example
emb = [get_vector_embeddings(chunk) for chunk in chunks]
vectors = np.array(emb)

# Create a FAISS index
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

# Function to perform a vector search
def vector_search(query_text, k=1):
    query_vector = get_vector_embeddings(query_text)
    distances, indices = index.search(
        np.array([query_vector]), k)
    return [(chunks[i], float(dist)) for dist, 
        i in zip(distances[0], indices[0])]

# Example search
user_query = "do we get free unicorn rides?"
search_results = vector_search(user_query)
print(f"Search results for {user_query}:", search_results)

Search results for do we get free unicorn rides?: [('At Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don\'t be surprised if you find a fairy tale book in the breakroom or a gnome guiding you to the restroom. Our culture', 0.3526668846607208)]


In [9]:
# Function to perform a vector search and then ask # GPT-3.5-turbo a question:
def search_and_chat(user_query, k=1):
  
  # Perform the vector search
  search_results = vector_search(user_query, k)
  print(f"Search results: {search_results}\n\n")

  prompt_with_context = f"""Context:{search_results}\
  Answer the question: {user_query}"""
  
  # Create a list of messages for the chat
  messages = [
      {"role": "system", "content": """Please answer the
      questions provided by the user. Use only the context
      provided to you to respond to the user, if you don't
      know the answer say \"I don't know\"."""},
      {"role": "user", "content": prompt_with_context},
  ]

  # Get the model's response
  response = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages)

  # Print the assistant's reply
  print(f"""Response: 
  {response.choices[0].message.content}""")

# Example search and chat
#search_and_chat("How often can I wear a costume to work?")

#search_and_chat("What is the weather today?")

search_and_chat("Benefits")

Search results: [("**4: Enchanted Benefits**\n\nOur commitment to your well-being extends to our magical benefits package. You'll enjoy a treasure chest of perks, including unlimited unicorn rides, a bottomless cauldron of coffee and potions, and access to our company library filled with spellbinding books. We also offer competitive health and dental plans, ensuring your physical well-being is as robust as your magical spirit.\n\n**5: Continuous Learning and Growth**", 0.3575068712234497)]


Response: 
  The benefits offered include unlimited unicorn rides, a bottomless cauldron of coffee and potions, access to a company library with spellbinding books, competitive health and dental plans, and a commitment to well-being.


In [10]:
# Save the index to a file
faiss.write_index(index, "my_index_file.index")

In [11]:
# Load the index from a file
index = faiss.read_index("my_index_file.index")

In [12]:
# Function to perform a vector search and then ask GPT-3.5-turbo a question:
def search_and_chat(user_query, k=1):
  
  # Perform the vector search
  search_results = vector_search(user_query, k)
  print(f"Search results: {search_results}\n\n")

  prompt_with_context = f"""Context:{search_results}\
  Answer the question: {user_query}"""
  
  # Create a list of messages for the chat
  messages = [
      {"role": "system", "content": """Please answer the
      questions provided by the user. Use only the context
      provided to you to respond to the user, if you don't
      know the answer say \"I don't know\"."""},
      {"role": "user", "content": prompt_with_context},
  ]

  # Get the model's response
  response = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages)

  # Print the assistant's reply
  print(f"""Response: 
  {response.choices[0].message.content}""")

In [13]:
search_and_chat("unicorn")

Search results: [('At Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don\'t be surprised if you find a fairy tale book in the breakroom or a gnome guiding you to the restroom. Our culture', 0.38127031922340393)]


Response: 
  Unicorn is a mythical creature known for its magical horn. Unicorns are often associated with purity, grace, and enchantment. In the provided context, "unicorn" is a reference to the company name "Unicorn Enterprises" and its associated unique and enchanting company culture.
